# Problem Statement

1.	Use Amazon Polly to convert the text “The sun does arise and make happy the skies. The merry bells ring to welcome the spring.” into a mp3 file using the male voice of Mathew. Then use Amazon Transcribe to convert the voice in the mp3 file back into text. Tell us what that text is. Does it exactly match the input text?
You will need to create Amazon S3 buckets for using Amazon Transcribe.

In [1]:
from __future__ import print_function
import boto3
from contextlib import closing
import time

# # Program to convert text to mp3 file using Polly

In [2]:
def polly_convert_txt_voice():
   client = boto3.client('polly')
   response = client.synthesize_speech(Text="The sun does arise and make happy the skies. The merry bells ring to welcome the spring.", 
                                       OutputFormat='mp3', VoiceId='Matthew')
   if "AudioStream" in response:
        with closing(response["AudioStream"]) as stream:
            data = stream.read()
            fo = open("Voice_Matthew.mp3", "wb")
            fo.write( data )
            fo.close()

## Calling above function

In [3]:
polly_convert_txt_voice()

## Function to convert the above mp3 stored in S3 to text using Transcribe.

In [4]:
def transcribe_convert_voice_txt():
    transcribe = boto3.client('transcribe')
    job_name = 'Assessment1_Ram_Job2'
    job_uri = "https://s3-us-west-2.amazonaws.com/awsmlbucket007/Voice_Matthew.mp3"
    transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': job_uri},
    MediaFormat='mp3',
    LanguageCode='en-US')
    
    while True:
        status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
            print("Not ready yet...")
        time.sleep(5)
    print(status)

## Calling the above function

In [5]:
transcribe_convert_voice_txt()

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Sun, 01 Jul 2018 18:15:33 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '1453', 'connection': 'keep-alive', 'x-amzn-requestid': 'be8e8526-7d5a-11e8-8059-0bf1f60f312b'}, 'HTTPStatusCode': 200, 'RetryAttempts': 0, 'RequestId': 'be8e8526-7d5a-11e8-8059-0bf1f60f312b'}, 'TranscriptionJob': {'CreationTime': datetime.datetime(2018, 7, 1, 13, 14, 31, 522000, tzinfo=tzlocal()), 'TranscriptionJobName': 'Assessment1_Ram_Job2', 'MediaSampleRateHertz': 22050, 'Transcript': {'TranscriptFileUri': 'https://s3.us-west-2.amazonaws.com/aws-transcribe-us-west-2-prod/345303837793/Assessment1_Ram_Job2/asrOutput.json?X-Amz-Security-Token=FQoDYXdzEDIaDFrOhkSKWaxVuc9eVyK3A104kyx24wC9wfopgEHDSIC9HQhSrAeWWQ9H4aYKGUMrwzX7ISGsLAuGrGSHEYPJEzKCrQu4dQfVlxg4KfsQxHvw%2BqPHptrOEinKWv9LD%2BchmQNl3%2BDhL6YfNexO9WMw0XJ14MPlDhCzex8vxuHTgb2Ioho3hzqo194IlWRXPBVNMzfZwlNtr4yALrpHEWrZVUR1skxEA5HE3ruG1b5X4qf0OA%2BdxvirecHnHKa2KjHGuvy0dWHkOmAZytHrmzvZNN2iPaS9BE